In [14]:
#Load Libraries
from datetime import date
import pyodbc
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from sportsreference.nba.teams import Teams
import pandas.io.sql as psql

In [4]:
#for development in azure notebooks (databricks uses spark behind the scenes and does not require this step)
sc = SparkContext('local')
spark = SparkSession(sc)

In [5]:
#Create Dataframe
DimTeam = pd.DataFrame()

#Load Team name and Abbreviation into Dataframe
for x in range(2000,2020):
    for team in Teams(x):
      DimTeam = DimTeam.append({'Name':team.name,'Abbreviation':team.abbreviation},ignore_index='true')
    x=x+1
 

In [6]:
#load into spark dataframe
DimTeamSparkDF = spark.createDataFrame(DimTeam)
#DimTeamSparkDF.createOrReplaceTempView("DimTeam")
columnsToHash = [x for x in DimTeamSparkDF.columns if str(x) not in ['Columns to ignore here']]
DimTeamSparkDF = DimTeamSparkDF.withColumn("RowValueHash", sha2(concat_ws("|", *columnsToHash), 256))
DimTeamSparkDF.createOrReplaceTempView("DimTeam")

In [9]:
#select * from df
sqlQuery = 'Select distinct * \
            from DimTeam \
            Order By name desc' 
spark.sql(sqlQuery).show()

+------------+--------------------+--------------------+
|Abbreviation|                Name|        RowValueHash|
+------------+--------------------+--------------------+
|         WAS|  Washington Wizards|fe8c73e1572efec74...|
|         VAN| Vancouver Grizzlies|65635413ec6148e75...|
|         UTA|           Utah Jazz|68194e2106b1a90ad...|
|         TOR|     Toronto Raptors|d8fe3a2f3f5b6f3f9...|
|         SEA| Seattle SuperSonics|7b3b4262ea607f082...|
|         SAS|   San Antonio Spurs|cfa7a0e82310cba17...|
|         SAC|    Sacramento Kings|c36dff5da5b0244fe...|
|         POR|Portland Trail Bl...|2ed836890bd3cfe85...|
|         PHO|        Phoenix Suns|2e22976f658527526...|
|         PHI|  Philadelphia 76ers|797c91749afd98acc...|
|         ORL|       Orlando Magic|0e1b7d04d74fb7dd3...|
|         OKC|Oklahoma City Thu...|0178a8324013b8c21...|
|         NYK|     New York Knicks|068f773b51179cf47...|
|         NOK|New Orleans/Oklah...|6b286487e33d5a2df...|
|         NOP|New Orleans Pelic

In [21]:
#Connect to SQL Database
server = 'mintdatabase.database.windows.net' 
database = 'NBADataMart' 
username = 'oren' 
password = 'Gpeqlx33j59o' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

#df = cursor.execute('Select * from DimTeam')
cursor = cnxn.cursor()
sql = "SELECT * FROM DimTeam"

df = psql.read_sql(sql, cnxn)
cnxn.close()



In [24]:
df

,TeamKey,TeamAbbreviation,TeamName,RecordCreateDate
